In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize
def test_empty_match_basic():
    """Test specific empty pattern case"""
    df = pd.DataFrame({
        'id': [1, 2, 3],
        'val': [10, 20, 30]
    })
    
    query = """
    SELECT *
    FROM df 
    MATCH_RECOGNIZE(
        ORDER BY id
        MEASURES MATCH_NUMBER() AS match_num
        PATTERN (())
        DEFINE x AS TRUE
    )
    """
    
    result = match_recognize(query, df)
    print(result)
    
    assert len(result) == 3
    assert list(result['match_num']) == [1, 2, 3]
test_empty_match_basic()

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM df MATCH_RECOGNIZE( ORDER BY id MEASURES MATCH_NUMBER() AS match_num PATTERN (()) DEFINE x AS TRUE );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='df')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='id', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='MATCH_NUMBER()', alias='match_num', metadata={'semantics': 'RUNNING'}, is_classifier=False, is_match_number=True)])
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='()', metadata={'variables': [], 'base_variables': [], 'empty_patt

Pattern value: '()'
Empty pattern detected - allowing any variable definition
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'match_num': 'FINAL'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Start state is accepting - pattern allows empty matches
DFA state 0 has transitions for variables: []
Built DFA with 1 states
  State 0: Accept, Variables: None, Excluded: None, Transitions: 
Find matches with all_rows=False, show_empty=True, include_unmatched=False
Starting match at index 0, state: 0
Found empty match at index 0 - start state is accepting
Starting match at index 1, state: 0
Found empty match at index 1 - start state is accepting
Starting match at index 2, state: 0
Found empty match at index 2 - start state is accepting
Find matches completed in 0.000080 seconds
   id  val  match_num  MATCH_NUMBER  IS_EMPTY_MATCH
0   1   10          1             1            True
1   2   20          2             2            True
2   3   30          3   

In [2]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
    SELECT id, name FROM employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES 
            salary AS current_salary,
            RUNNING SUM(salary) AS running_sum,
            MATCH_NUMBER() AS match_num
        ALL ROWS PER MATCH
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT id, name FROM employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS current_salary, RUNNING SUM(salary) AS running_sum, MATCH_NUMBER() AS match_num ALL ROWS PER MATCH PATTERN (A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=id, metadata={}), SelectItem(expression=name, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='employees')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted M

Pattern value: 'A+'
Defined variables: {'A'}
Current base variables: {'A'}
Pattern value: 'A+'
Defined variables: {'A'}
Current base variables: {'A'}
Raw rows_per_match text: ALLROWSPERMATCH
Using rows_per_match mode: RowsPerMatch.ALL_ROWS_SHOW_EMPTY
Measure semantics: {'current_salary': 'RUNNING', 'running_sum': 'RUNNING', 'match_num': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ALL_ROWS_SHOW_EMPTY, all_rows=True
Creating transition for variable 'A' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
DFA state 0 has transitions for variables: ['A']
  Added transition from state 0 to 1 for variable A
DFA state 1 has transitions for variables: ['A']
  Added transition from state 1 to 1 for variable A
Built DFA with 2 states
  State 0: Non-accept, Variables: A, Excluded: None, Transitions: A → 1
  State 1: Accept, Variables: A, Excluded: None, Transitions: A → 1
Find matches with all_rows=True, show_empty=True, include_unmatched=False
Startin

In [3]:
import pandas as pd
from src.executor.match_recognize import match_recognize
# Use an absolute import for match_recognize.

query = """
    SELECT id, name FROM employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
        {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT id, name FROM employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS avg_salary PATTERN (A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=id, metadata={}), SelectItem(expression=name, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='employees')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='salary', alias='avg_salary', metad

Pattern value: 'A+'
Defined variables: {'A'}
Current base variables: {'A'}
Pattern value: 'A+'
Defined variables: {'A'}
Current base variables: {'A'}
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'avg_salary': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Creating transition for variable 'A' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
DFA state 0 has transitions for variables: ['A']
  Added transition from state 0 to 1 for variable A
DFA state 1 has transitions for variables: ['A']
  Added transition from state 1 to 1 for variable A
Built DFA with 2 states
  State 0: Non-accept, Variables: A, Excluded: None, Transitions: A → 1
  State 1: Accept, Variables: A, Excluded: None, Transitions: A → 1
Find matches with all_rows=False, show_empty=True, include_unmatched=False
Starting match at index 0, state: 0
Testing row 0, data: {'id': 1, 'name': 'Alice', 'department': 'Sales', 'region': 'West',

In [4]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);

    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS current_salary, RUNNING SUM(salary) AS running_sum, MATCH_NUMBER() AS match_num ALL ROWS PER MATCH PATTERN (A {- B+ -} C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURE

Pattern value: 'A {- B+ -} C+'
Defined variables: {'B', 'A', 'C'}
Current base variables: {'B', 'A', 'C'}
Pattern value: 'A {- B+ -} C+'
Defined variables: {'B', 'A', 'C'}
Current base variables: {'B', 'A', 'C'}
Raw rows_per_match text: ALLROWSPERMATCH
Using rows_per_match mode: RowsPerMatch.ALL_ROWS_SHOW_EMPTY
Measure semantics: {'current_salary': 'RUNNING', 'running_sum': 'RUNNING', 'match_num': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ALL_ROWS_SHOW_EMPTY, all_rows=True
Creating transition for variable 'A' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
Creating transition for variable 'B' with condition: 'salary < 1000'
Compiled condition: 'salary < 1000' -> 'salary < 1000.0'
Creating transition for variable 'C' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
DFA state 0 has transitions for variables: ['A']
  Added transition from state 0 to 1 for variable A
DFA state 1 has transitions for 

In [5]:
query = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ALL ROWS PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);
"""

In [6]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        CLASSIFIER(A) AS is_a_var,
        CLASSIFIER(C) AS is_c_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ALL ROWS PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);
"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY department, region ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, CLASSIFIER(A) AS is_a_var, CLASSIFIER(C) AS is_c_var, salary AS current_salary, RUNNING SUM(salary) AS running_sum ALL ROWS PER MATCH PATTERN (A {- B+ -} C+) DEFINE A AS salary > 1000, B AS salary < 1000, C AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:

Pattern value: 'A {- B+ -} C+'
Defined variables: {'B', 'A', 'C'}
Current base variables: {'B', 'A', 'C'}
Pattern value: 'A {- B+ -} C+'
Defined variables: {'B', 'A', 'C'}
Current base variables: {'B', 'A', 'C'}
Raw rows_per_match text: ALLROWSPERMATCH
Using rows_per_match mode: RowsPerMatch.ALL_ROWS_SHOW_EMPTY
Measure semantics: {'pattern_var': 'RUNNING', 'is_a_var': 'RUNNING', 'is_c_var': 'RUNNING', 'current_salary': 'RUNNING', 'running_sum': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ALL_ROWS_SHOW_EMPTY, all_rows=True
Creating transition for variable 'A' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
Creating transition for variable 'B' with condition: 'salary < 1000'
Compiled condition: 'salary < 1000' -> 'salary < 1000.0'
Creating transition for variable 'C' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
DFA state 0 has transitions for variables: ['A']
  Added transition from state 0 to 1

In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different departments to test partition behavior
data = [
    # Sales department - First row has high salary
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    
    # Marketing department - Last row has high salary
    {"id": 5, "name": "Eve", "department": "Marketing", "region": "East", "hire_date": "2021-01-01", "salary": 900},
    {"id": 6, "name": "Frank", "department": "Marketing", "region": "East", "hire_date": "2021-01-02", "salary": 950},
    {"id": 7, "name": "Grace", "department": "Marketing", "region": "East", "hire_date": "2021-01-03", "salary": 980},
    {"id": 8, "name": "Henry", "department": "Marketing", "region": "East", "hire_date": "2021-01-04", "salary": 1200},
    
    # IT department - All rows have high salary
    {"id": 9, "name": "Ivy", "department": "IT", "region": "North", "hire_date": "2021-01-01", "salary": 1500},
    {"id": 10, "name": "Jack", "department": "IT", "region": "North", "hire_date": "2021-01-02", "salary": 1600},
    {"id": 11, "name": "Kate", "department": "IT", "region": "North", "hire_date": "2021-01-03", "salary": 1700},
    {"id": 12, "name": "Leo", "department": "IT", "region": "North", "hire_date": "2021-01-04", "salary": 1800},
    
    # HR department - No rows have high salary
    {"id": 13, "name": "Mike", "department": "HR", "region": "South", "hire_date": "2021-01-01", "salary": 950},
    {"id": 14, "name": "Nina", "department": "HR", "region": "South", "hire_date": "2021-01-02", "salary": 980},
    {"id": 15, "name": "Oscar", "department": "HR", "region": "South", "hire_date": "2021-01-03", "salary": 990},
    {"id": 16, "name": "Pam", "department": "HR", "region": "South", "hire_date": "2021-01-04", "salary": 995},
]

df = pd.DataFrame(data)

print("Testing Pattern Anchors\n")

# Test 1: Start anchor (^) - Should match patterns starting at the beginning of a partition
query_start_anchor = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 1: Start Anchor (^) - Should only match departments where first employee has salary > 1000")
output_df = match_recognize(query_start_anchor, df)
print(output_df)
print("\n")

# Test 2: End anchor ($) - Should match patterns ending at the end of a partition
query_end_anchor = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 2: End Anchor ($) - Should only match departments where last employee has salary > 1000")
output_df = match_recognize(query_end_anchor, df)
print(output_df)
print("\n")

# Test 3: Both anchors (^$) - Should match patterns spanning the entire partition
query_both_anchors = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 3: Both Anchors (^$) - Should only match departments where ALL employees have salary > 1000")
output_df = match_recognize(query_both_anchors, df)
print(output_df)
print("\n")

# Test 4: Start anchor with ALL ROWS PER MATCH to see the actual matched rows
query_start_all_rows = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 4: Start Anchor (^) with ALL ROWS PER MATCH - Shows matched rows")
output_df = match_recognize(query_start_all_rows, df)
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY department ORDER BY hire_date MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (^A+) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='CLASSIFIER()', alias='pattern_var', metadata={'semanti

Testing Pattern Anchors

Test 1: Start Anchor (^) - Should only match departments where first employee has salary > 1000
Pattern value: '^A+'
Defined variables: {'A'}
Current base variables: {'A'}


DEBUG:src.parser.match_recognize_extractor:PATTERN clause validated successfully: ^A+
DEBUG:src.parser.match_recognize_extractor:Extracted Pattern: PatternClause(pattern='^A+', metadata={'variables': ['A+'], 'base_variables': ['A']})
DEBUG:src.parser.match_recognize_extractor:Extracted DEFINE: DefineClause(definitions=[Define(variable='A', condition='salary > 1000')])
DEBUG:src.parser.match_recognize_extractor:Updated Pattern tokens: {'variables': ['A+'], 'base_variables': ['A']}
DEBUG:src.parser.match_recognize_extractor:PATTERN clause validated successfully: ^A+
DEBUG:src.parser.match_recognize_extractor:Extracted variables from measure 'CLASSIFIER()': []
DEBUG:src.parser.match_recognize_extractor:Extracted variables from measure 'MATCH_NUMBER()': []
DEBUG:src.parser.match_recognize_extractor:Pattern variables: {'A'}
DEBUG:src.parser.match_recognize_extractor:Referenced variables: set()
DEBUG:src.parser.match_recognize_extractor:Defined variables: {'A'}
DEBUG:src.parser.match_recogni

Pattern value: '^A+'
Defined variables: {'A'}
Current base variables: {'A'}
Raw rows_per_match text: ONEROWPERMATCH
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'pattern_var': 'FINAL', 'match_num': 'FINAL'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Creating transition for variable 'A' with condition: 'salary > 1000'
Compiled condition: 'salary > 1000' -> 'salary > 1000.0'
DFA state 0 has transitions for variables: ['A']
  Added transition from state 0 to 1 for variable A
DFA state 1 has transitions for variables: ['A']
  Added transition from state 1 to 1 for variable A
Built DFA with 2 states
  State 0: Non-accept, Variables: A, Excluded: None, Transitions: A → 1
  State 1: Accept, Variables: A, Excluded: None, Transitions: A → 1
Find matches with all_rows=False, show_empty=True, include_unmatched=False
Starting match at index 0, state: State 0 (start anchor ^) vars: ['A']
Testing row 0, data: {'id': 13, 'name': 'Mike', 'department': 'HR'

In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")

# Test 1: Basic PERMUTE - Match any order of A, B, C
query_basic_permute = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order")
output_df = match_recognize(query_basic_permute, df)
print(output_df)
print("\n")

# Test 2: PERMUTE with Quantifier
query_permute_quantifier = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        FIRST(A.value) AS first_a_value,
        LAST(C.value) AS last_c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C)+)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 2: PERMUTE with Quantifier - Should match one or more occurrences of permutations")
output_df = match_recognize(query_permute_quantifier, df)
print(output_df)
print("\n")

# Test 3: PERMUTE with ALL ROWS PER MATCH
query_permute_all_rows = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        RUNNING LAST(A.value) AS running_a_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 3: PERMUTE with ALL ROWS PER MATCH - Shows all matched rows")
output_df = match_recognize(query_permute_all_rows, df)
print(output_df)
print("\n")

# Test 4: PERMUTE with Subset Variables
query_permute_subset = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        X.value AS x_value,
        Y.value AS y_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    SUBSET
        X = (A, B),
        Y = (B, C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 4: PERMUTE with Subset Variables - Using subset groupings")
output_df = match_recognize(query_permute_subset, df)
print(output_df)
print("\n")

# Test 5: Nested PERMUTE patterns
query_nested_permute = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, PERMUTE(B, C)))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 5: Nested PERMUTE - Testing nested permutation patterns")
output_df = match_recognize(query_nested_permute, df)
print(output_df)
print("\n")

# Test 6: PERMUTE with Complex Conditions
query_permute_complex = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS start_value,
        B.value AS middle_value,
        C.value AS end_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start' AND A.value < NEXT(A.value),
        B AS event_type = 'middle' AND B.value > PREV(B.value),
        C AS event_type = 'end' AND C.value > FIRST(A.value)
);
"""

print("Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions")
output_df = match_recognize(query_permute_complex, df)
print(output_df)
print("\n")

# Test 7: PERMUTE with Edge Cases
query_permute_edge_cases = """
SELECT * FROM orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        LAST(B.value) AS last_b_value,
        FIRST(C.value) AS first_c_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B?, C?))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 7: PERMUTE with Edge Cases - Testing optional elements")
output_df = match_recognize(query_permute_edge_cases, df)
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS a_value, B.value AS b_value, C.value AS c_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start', B AS event_type = 'middle', C AS event_type = 'end' );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted 

Testing PERMUTE Patterns

Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order
Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
Raw rows_per_match text: ONEROWPERMATCH
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'pattern_var': 'FINAL', 'match_num': 'FINAL', 'a_value': 'RUNNING', 'b_value': 'RUNNING', 'c_value': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Creating transition for variable 'A' with condition: 'event_type = 'start''
Creating transition for variable 'B' with condition: 'event_type = 'middle''
Creating transition for variable 'C' with condition: 'event_type = 'end''
Creating transition for variable 'A' with condition: 'event_type = 'start''
Creating transition for variable 'C' with condition: 'event_type = 'end''
Creating transition for variable 'B' with condition: 'event_type = 'middle''
Creating transition for variable 'B' with condition: 'event_type = 'middle''
Creating tra

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES CLASSIFIER() AS pattern_var, MATCH_NUMBER() AS match_num, A.value AS start_value, B.value AS middle_value, C.value AS end_value ONE ROW PER MATCH PATTERN (PERMUTE(A, B, C)) DEFINE A AS event_type = 'start' AND A.value < NEXT(A.value), B AS event_type = 'middle' AND B.value > PREV(B.value), C AS event_type = 'end' AND C.value > FIRST(A.value) );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='ste

Pattern value: 'PERMUTE(A, B, C)'
Pattern value: 'PERMUTE(A, B, C)'
Raw rows_per_match text: ONEROWPERMATCH
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'pattern_var': 'FINAL', 'match_num': 'FINAL', 'start_value': 'RUNNING', 'middle_value': 'RUNNING', 'end_value': 'RUNNING'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Creating transition for variable 'A' with condition: 'event_type = 'start' AND A.value < NEXT(A.value)'
Syntax error parsing: 'event_type = 'start' AND A.value < NEXT(A.value)' (processed as 'event_type = "start" AND A.value < get_next_value("A.value", 1, ctx)')
Error details: invalid syntax (<unknown>, line 1)
Creating transition for variable 'B' with condition: 'event_type = 'middle' AND B.value > PREV(B.value)'
Syntax error parsing: 'event_type = 'middle' AND B.value > PREV(B.value)' (processed as 'event_type = "middle" AND B.value > get_prev_value("B.value", 1, ctx)')
Error details: invalid syntax (<unknown>, line 1)
Creati